# Training

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [3]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(32, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(64, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 32)           896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 32)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 64)             2112      ['max_pooling2d[0][0]']  

## fit 함수 사용

$$
\Large{
model.compile() \rightarrow model.fit()
}
$$

#### compile의 입력값

- optimizer='rmsprop' : Optimizer 
- loss=None : Loss function
- metrics=None : Metrics
- loss_weights=None : loss가 여러 개인 경우 각 로스마다 다르게 중요도를 설정 할 수 있다. 


loss 지정하는 방법

1. `tf.keras.losses`이용

In [5]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

In [6]:
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

2. 사용자 정의함수로 loss 지정

In [7]:
# 사용자 정의 loss : 입력으로 정답값, 예측값의 순서로 꼭 받아야함
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [8]:
model.compile(optimizer=opt, loss=custom_loss, metrics=["accuracy"])

이때 loss를 여러 개를 입력할 수도, 각각의 중요도를 조절할 수도 있음

In [9]:
#여러 개의 Loss : 리스트 형태로 입력
model.compile(optimizer=opt, loss=[loss, custom_loss], metrics=["accuracy"])

In [10]:
#여러 개의 Loss + loss weights
# 각각의 loss에 대한 중요도 조절
# 각 loss에 wieghts를 곱해서 더한 값을 최종 loss로 사용
model.compile(optimizer=opt, loss=[loss, custom_loss], loss_weights=[0.7, 0.3], metrics=["accuracy"])

3. 텍스트로 loss 지정

In [36]:
loss = "categorical_crossentropy" # 이렇게 텍스트로 가능한 함수도 있음.

model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

metrics 지정하는 방법

1. `tf.keras.metrics` 이용

In [37]:
# mertircs에도 여러 개를 넣을 수 있음
acc = tf.keras.metrics.Accuracy()
auc = tf.keras.metrics.AUC()

In [38]:
model.compile(optimizer=opt, loss=loss, metrics=[acc, auc])

2. 사용자 정의 함수로 지정

In [52]:
def custom_metric(y_true, y_pred):
    
    true = tf.argmax(y_true, axis=-1)
    pred = tf.argmax(y_pred, axis=-1)
    
    return tf.reduce_sum(tf.cast(tf.equal(true, pred), tf.int32))


In [53]:
model.compile(optimizer=opt, loss=loss, metrics=[custom_metric])

In [54]:
model.compile(optimizer=opt, loss=loss, metrics=["accuracy", custom_metric])

### fit의 입력값

- x=None 
- y=None
- batch_size=None
- epochs=1
- verbose='auto' : 학습과정 출력문의 모드
- callbacks=None : Callback 함수
- validation_split=0.0 : 입력데이터의 일정 부분을 Validation 용 데이터로 사용함
- validation_data=None : Validation 용 데이터 
- shuffle=True : 입력값을 Epoch 마다 섞는다. 
- class_weight=None : 클래스 별로 다른 중요도를 설정한다. 
- ...

In [55]:
hist = model.fit(train_x, 
                 train_y,
                 epochs=1, 
                 batch_size=128, 
                 validation_split=0.3,
                 verbose=1
                )

274/274 [==============================] - 20s 72ms/step - loss: 1.8701 - accuracy: 0.3338 - custom_metric: 0.0000e+00 - val_loss: 1.8292 - val_accuracy: 0.3471 - val_custom_metric: 0.0000e+00


### Callback 함수 활용하기 

 - Callback 함수를 활용하면, fit() 함수가 돌아가는 와중에도 특정한 주기로 원하는 코드를 실행 시킬 수 있음. 
 
 ex> 학습이 진행되면, Learning rate를 점점 줄여서 더 세밀하게 모델의 웨이트가 조정 될 수 있도록 하고 싶다. 

 `tf.keras.callbacks.LearningRateScheduler` : 입력값으로 epoch와 lr을 받음

In [15]:
# epoch이 10을 초과할 때마다 lr을 감소시켜 안정적으로 수렴하도록 도움을 줌
def scheduler(epoch, lr):
    if epoch > 10:
        return lr * (0.9**(epoch - 10))
    else: 
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
hist = model.fit(train_x, 
                 train_y,
                 epochs=20, 
                 batch_size=128, 
                 validation_split=0.3,
                 verbose=1,
                 callbacks=[lr_scheduler],
                )

Epoch 1/20
274/274 [==============================] - 14s 52ms/step - loss: 1.7438 - accuracy: 0.0994 - val_loss: 1.6223 - val_accuracy: 0.1005 - lr: 0.1180
Epoch 2/20
274/274 [==============================] - 14s 51ms/step - loss: 1.7269 - accuracy: 0.1006 - val_loss: 1.6618 - val_accuracy: 0.0979 - lr: 0.1180
Epoch 3/20
274/274 [==============================] - 15s 54ms/step - loss: 1.6960 - accuracy: 0.0979 - val_loss: 1.6293 - val_accuracy: 0.1014 - lr: 0.1180
Epoch 4/20
274/274 [==============================] - 15s 54ms/step - loss: 1.6753 - accuracy: 0.0990 - val_loss: 1.6337 - val_accuracy: 0.0999 - lr: 0.1180
Epoch 5/20
274/274 [==============================] - 15s 54ms/step - loss: 1.7164 - accuracy: 0.1003 - val_loss: 1.6252 - val_accuracy: 0.1014 - lr: 0.1180
Epoch 6/20
274/274 [==============================] - 15s 56ms/step - loss: 1.6642 - accuracy: 0.1019 - val_loss: 1.6376 - val_accuracy: 0.1025 - lr: 0.1180
Epoch 7/20
274/274 [==============================] - 16s 

In [ ]:
# 위와 같은 callback이나 earlystoping, history를 남기는 callback등 다양한 callback 함수를 사용함.